In [1]:
# -*- coding: utf-8 -*-
import cv2 # opencv 사용
import numpy as np

def region_of_interest(img, vertices, color3=(255,255,255), color1=255): # 2. ROI 셋팅

    mask = np.zeros_like(img) # mask = img와 같은 크기의 빈 이미지
    
    if len(img.shape) > 2: # Color 이미지(3채널)라면 :
        color = color3 
    else: # 흑백 이미지(1채널)라면 :
        color = color1
        
    # vertices에 정한 점들로 이뤄진 다각형부분(ROI 설정부분)을 color로 채움 
    cv2.fillPoly(mask, vertices, color)
    
    # 이미지와 color로 채워진 ROI를 합침
    ROI_image = cv2.bitwise_and(img, mask)
    return ROI_image

def mark_img(img, blue_threshold=200, green_threshold=200, red_threshold=200): # 1. 흰색 차선 찾기

    #  BGR 제한 값 (기준)
    bgr_threshold = [blue_threshold, green_threshold, red_threshold]

    # BGR 제한 값(기준)보다 작으면 검은색으로
    thresholds = (image[:,:,0] < bgr_threshold[0]) \
                | (image[:,:,1] < bgr_threshold[1]) \
                | (image[:,:,2] < bgr_threshold[2])
    mark[thresholds] = [0,0,0] # 검정색으로 
    return mark

cap = cv2.VideoCapture('solidWhiteRight.mp4') # 동영상 불러오기 (960 x 540)
#cap = cv2.VideoCapture('testvideo.mp4') # 동영상 불러오기 (1920 x 1080)

while(cap.isOpened()):
    ret, image = cap.read()
    image = cv2.resize(image, (640, 360)) # window 사이즈 조정 
    
    height, width = image.shape[:2] # 이미지 높이, 너비

    # 사다리꼴 모형의 Points
    vertices = np.array([[(50,height),(width/2-45, height/2+60), (width/2+45, height/2+60), (width-50,height)]], dtype=np.int32)
    # 관심영역 
    roi_img = region_of_interest(image, vertices, (0,0,255)) # vertices에 정한 점들 기준으로 ROI 이미지 생성 (= 차선부분만 관심영역)
                                                                # (0,0,255)를 더해줌으로써 관심영역 전체를 빨간색으로 출력 
    
    mark = np.copy(roi_img) # roi_img를 그대로 복사
    mark = mark_img(roi_img) # 원래 흰색아닌 부분을 모두 검정색으로 변환 
    
    #cv2.imshow('results',roi_img) # 관심영역 전체를 빨간색으로 출력 
    cv2.imshow('results',mark) # 원래 흰색이었던 차선만 빨간색으로 추출 (곡선 추출도 당연히 가능)
    
    '''
    # 흰색 차선 검출한 부분을 원본 image에 overlap 하기
    color_thresholds = (mark[:,:,0] == 0) & (mark[:,:,1] == 0) & (mark[:,:,2] > 200)
    image[color_thresholds] = [0,0,255]

    #cv2.imshow('results',image) # 원본 동영상과 합쳐진 이미지 출력
    '''
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# Release
cap.release()
cv2.destroyAllWindows()